## Deutsch's Algorithm

### Overview
Deutsch's Algorithm is a simple quantum algorithm that determines whether a given function \( $f: \{0, 1\} \rightarrow \{0, 1\}$ \) is constant or balanced. The algorithm requires only one query to the function, whereas a classical algorithm would require two.

### Sequential Code

In [2]:
%%writefile deutsch-sequential.c 
#include <stdio.h>

// Define the function f: {0, 1} -> {0, 1}
int f(int x) {
    // Example function: f(0) = 0, f(1) = 1 (balanced)
    return x;
}

int deutschAlgorithm() {
    // Evaluate f(0) and f(1)
    int f0 = f(0);
    int f1 = f(1);

    // Check if f is constant or balanced
    if (f0 == f1) {
        return 0; // Constant function
    } else {
        return 1; // Balanced function
    }
}

int main(int argc, char **argv) 
{
    int result = deutschAlgorithm();

    if (result == 0) {
        printf("The function is constant.\n");
    } else {
        printf("The function is balanced.\n");
    }

    return 0;
}

Overwriting deutsch-sequential.c


In [3]:
!gcc deutsch-sequential.c -o deutsch-sequential

In [4]:
!./deutsch-sequential

The function is balanced.


### CUDA

In [5]:
%%writefile deutsch-cuda.cu
#include <stdio.h>
#include <cuda.h>

// Define the function f: {0, 1} -> {0, 1}
__device__ int f(int x) {
    // Example function: f(0) = 0, f(1) = 1 (balanced)
    return x;
}

// CUDA kernel to evaluate f(0) and f(1) in parallel
__global__ void deutsch_kernel(int *d_results) {
    int tid = threadIdx.x; // Thread ID (0 or 1)
    d_results[tid] = f(tid); // Evaluate f(x) for x = tid
}

int main(int argc, char **argv) {

    int h_results[2]; // Host results array
    int *d_results;   // Device results array

    // Allocate memory on the device
    cudaMalloc((void **)&d_results, 2 * sizeof(int));

    // Launch the kernel with 2 threads (one for each input)
    deutsch_kernel<<<1, 2>>>(d_results);

    // Copy the results back to the host
    cudaMemcpy(h_results, d_results, 2 * sizeof(int), cudaMemcpyDeviceToHost);

    // Check if the function is constant or balanced
    if (h_results[0] == h_results[1]) {
        printf("The function is constant.\n");
    } else {
        printf("The function is balanced.\n");
    }

    // Free device memory
    cudaFree(d_results);

    return 0;
}

Overwriting deutsch-cuda.cu


In [6]:
!nvcc deutsch-cuda.cu -o deutsch-cuda

In [7]:
!./deutsch-cuda

The function is balanced.


### CUDA-Q `Python`

In [9]:
%%writefile deutsch-cudaq.py
import cudaq
import numpy as np
from typing import List

cudaq.set_target("nvidia")

# Here we input the values of [f(0), f(1)] which allows us to represent constant or balanced functions.
fx = [0, 1]

# Let us now code up the circuit shown above following the state Psi after each step.
qubit_count = 2

@cudaq.kernel
def kernel(fx: List[int]):
    qubit_0 = cudaq.qubit()
    qubit_1 = cudaq.qubit()

    # Psi 0
    x(qubit_1)

    # Psi 1
    h(qubit_0)
    h(qubit_1)

    # Psi 2 - oracle
    if fx[0] == 1:
        x.ctrl(qubit_0, qubit_1)
        x(qubit_1)

    if fx[1] == 1:
        x.ctrl(qubit_0, qubit_1)

    # Psi 3
    h(qubit_0)

    # Measure the qubit to yield if the function is constant or balanced.
    mz(qubit_0)

print(cudaq.draw(kernel, fx))

result = cudaq.sample(kernel, fx, shots_count=1)

if np.array(result)[0] == '0':
    print('The function is constant.')
elif np.array(result)[0] == '1':
    print('The function is balanced.')

Overwriting deutsch-cudaq.py


In [10]:
!python deutsch-cudaq.py

     ╭───╮          ╭───╮
q0 : ┤ h ├───────●──┤ h ├
     ├───┤╭───╮╭─┴─╮╰───╯
q1 : ┤ x ├┤ h ├┤ x ├─────
     ╰───╯╰───╯╰───╯     

The function is balanced.


### CUDA-Q `C++`

In [16]:
%%writefile deutsch-cudaq.cpp
//TODO
#include <cudaq.h>
#include <iostream>
#include <vector>

template <std::size_t N>
struct deutschAlgorithm {
  auto operator()() __qpu__ {

  cudaq::qarray<N> q;

  h(q[0]);  // Apply Hadamard gate to qubit 0
  x(q[1]);  // Apply X gate to qubit 1
  h(q[1]);  // Apply Hadamard gate to qubit 1

  x(q[0]);  // Apply X gate to qubit 0 if the function is balanced
    
  h(q[0]);  // Apply Hadamard gate to qubit 0 again
        
  auto result = mz(q[0]);

    // Output the result
    if (result == 0) {
        std::cout << "The function is constant." << std::endl;
    } else {
        std::cout << "The function is balanced." << std::endl;
    }
   
  }
};

int main(int argc, char **argv) {

    auto kernel = deutschAlgorithm<2>{};

    return 0;
}

Overwriting deutsch-cudaq.cpp


In [17]:
!nvq++ deutsch-cudaq.cpp -o deutsch-cudaq

In [18]:
!./deutsch-cudaq

## Clear the Memory

Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)